In [3]:
import collections
import os
with open(os.path.join('data','combinedText.txt'), encoding="utf-8", errors='ignore') as f:
    words = ' '.join(f.readlines())
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary
d1, d2 = build_dataset(words)
print(d1, d2)

{' ': 0, 'e': 1, 't': 2, 'o': 3, 'a': 4, 'h': 5, 'n': 6, 'i': 7, 's': 8, 'l': 9, 'r': 10, 'd': 11, '.': 12, 'u': 13, 'w': 14, 'y': 15, 'm': 16, 'c': 17, 'g': 18, 'b': 19, 'k': 20, 'p': 21, ',': 22, 'f': 23, 'v': 24, "'": 25, '-': 26, 'x': 27, '’': 28, 'j': 29, '“': 30, '”': 31, 'z': 32, 'q': 33, '…': 34, ':': 35, ';': 36, '1': 37, '5': 38, '6': 39, '(': 40, ')': 41, '2': 42, '9': 43, '8': 44, '3': 45, '/': 46, '4': 47} {0: ' ', 1: 'e', 2: 't', 3: 'o', 4: 'a', 5: 'h', 6: 'n', 7: 'i', 8: 's', 9: 'l', 10: 'r', 11: 'd', 12: '.', 13: 'u', 14: 'w', 15: 'y', 16: 'm', 17: 'c', 18: 'g', 19: 'b', 20: 'k', 21: 'p', 22: ',', 23: 'f', 24: 'v', 25: "'", 26: '-', 27: 'x', 28: '’', 29: 'j', 30: '“', 31: '”', 32: 'z', 33: 'q', 34: '…', 35: ':', 36: ';', 37: '1', 38: '5', 39: '6', 40: '(', 41: ')', 42: '2', 43: '9', 44: '8', 45: '3', 46: '/', 47: '4'}


In [4]:
import tensorflow as tf
vocab_size = len(d1)
n_input = 3
# number of units in RNN cell
n_hidden = 512
# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 1-layer LSTM with n_hidden units.
    rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
RNN(X, weights, biases)

NameError: name 'X' is not defined